In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [34]:
#calculate flux coming from one defined source
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
echan = 0
#detectors = np.array(['n3', 'n5', 'n0', 'n9'])

day = 140611
#burst_ra = 60.68
#burst_dec = 32.77
binning = 1

#start_time = 7.9
#end_time = 8.2


#burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

#sat_time = rf.poshist(day)[0]

#plot_time_sat_date = calc.met_to_date(sat_time)[0]
#plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

#mpl.rcParams['axes.color_cycle'] = ['g', 'r', 'c', 'm', 'y', 'k']

detec_residuals = []
detec_counts = []

for i in range(0, len(detectors)):
    #print detectors[i]
    #ang_data = calc.burst_ang_bin(detectors[i], day, burst_ra, burst_dec)
    #det_ang = ang_data[0]
    
    #det_ang_eff = calc.ang_eff(det_ang, echan)[0]
    #sat_time = det_data[1]
    
    ctime_data = calc.curve_fit_plots(day, detectors[i], echan)
    det_residuals = ctime_data[0]
    det_counts = ctime_data[1]
    det_fit = ctime_data[2]
    plot_time_bin = ctime_data[10]
    plot_time_sat = ctime_data[11]
    
    #eff_residuals = det_residuals*det_ang_eff
    #eff_counts = det_counts*det_ang_eff
    
    binned_res = det_residuals[0::binning]
    binned_counts = det_counts[0::binning]
    binned_times_bin = plot_time_bin[int(binning/2)::binning]
    binned_times_sat = plot_time_sat[int(binning/2)::binning]
    
    for j in range(1, binning):
        if len(binned_res) != len(det_residuals[j::binning]):
            binned_res = np.delete(binned_res, -1)
            binned_counts = np.delete(binned_counts, -1)
        binned_res = np.add(binned_res, det_residuals[j::binning])
        binned_counts = np.add(binned_counts, det_counts[j::binning])
    
    if len(binned_res) < len(binned_times_bin):
        binned_times_bin = np.delete(binned_times_bin, -1)
        binned_times_sat = np.delete(binned_times_sat, -1)
    
    detec_residuals.append(binned_res)
    detec_counts.append(binned_counts)
    
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    #plt.plot(binned_times_bin, binned_res, label = detectors[i])
    
    
    
detec_residuals = np.array(detec_residuals)
detec_counts = np.array(detec_counts)
#plot_residuals = np.squeeze(np.sum(plot_residuals, axis = 0))
#plot_counts = np.squeeze(np.sum(plot_counts, axis = 0))
    
#print plot_time_bin
#print tot_residuals

In [39]:
plot_detector = 'n5'

if plot_detector == 'n0':
    index = 0
    plot_residuals = detec_residuals[0] - 0.5*detec_residuals[8] - 0.5*detec_residuals[11]
elif plot_detector == 'n1':
    index = 1
    plot_residuals = detec_residuals[1] - 0.5*detec_residuals[8] - 0.5*detec_residuals[11]
elif plot_detector == 'n2':
    index = 2
    plot_residuals = detec_residuals[2] - detec_residuals[8]
elif plot_detector == 'n3':
    index = 3
    plot_residuals = detec_residuals[3] - 0.5*detec_residuals[10] - 0.5*detec_residuals[11]
elif plot_detector == 'n4':
    index = 4
    plot_residuals = detec_residuals[4] - detec_residuals[10]
elif plot_detector == 'n5':
    index = 5
    plot_residuals = detec_residuals[5] - detec_residuals[11]
elif plot_detector == 'n6':
    index = 6
    plot_residuals = detec_residuals[6] - 0.5*detec_residuals[2] - 0.5*detec_residuals[5]
elif plot_detector == 'n7':
    index = 7
    plot_residuals = detec_residuals[7] - 0.5*detec_residuals[2] - 0.5*detec_residuals[5]
elif plot_detector == 'n8':
    index = 8
    plot_residuals = detec_residuals[8] - detec_residuals[2]
elif plot_detector == 'n9':
    index = 9
    plot_residuals = detec_residuals[9] - 0.5*detec_residuals[4] - 0.5*detec_residuals[5]
elif plot_detector == 'na':
    index = 10
    plot_residuals = detec_residuals[10] - detec_residuals[4]
elif plot_detector == 'nb':
    index = 11
    plot_residuals = detec_residuals[11] - detec_residuals[5]


plt.plot(binned_times_bin, plot_residuals, 'b-')
plt.plot(binned_times_bin, detec_residuals[index], 'r-')
plt.grid()
plt.legend()

plt.show()

No labeled objects found. Use label='...' kwarg on individual plots.
